<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/DzDatascience/%D0%94%D0%97_UltraPro_20_%D0%BF%D0%B8%D1%81%D0%B0%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дорогой студент!

В домашнем задании Ultra Pro занятия по обработке тектсов с помощью НС мы ставим задачу распознать уже не 6, как ранее, а целых 20 русских писателей! Это подразумевает и больший размер базы для обучения соответственно. Ячейка для скачивания базы уже включена в ноутбук задания.


 В задании необходимо выполнить следующие пункты:

  1. Загрузить саму базу по ссылке и подговить файлы базы для обработки.
  2. Создать обучающую и проверочную выборки, обратив особое внимание на балансировку базы: количество примеров каждого класса должно быть примерно одного порядка. При этом для разбивки необходимо применить цикл. Проверочная выборка должна быть 20% от общей выборки.
  3. Подготовьте выборки для обучения и обучите сеть. Добейтесь результата точности сети не менее 95% на проверочной выборке модели Bag of Words и 75-80% - для модели Embedding.
   


## Загрузка данных

In [ ]:
import gdown                                      # Подключим функцию gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/20writers.zip', None, quiet=True)      # Скачивание файла

'20writers.zip'

In [ ]:
# Работа с массивами данных
import numpy as np

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Функции операционной системы
import os

# Работа со временем
import time

# Регулярные выражения
import re

# Запись в файлы и чтение из файлов структур данных Python
import pickle

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

Команды, именуемые «магическими» предназначены для выполнения кода непосредственно в консоли или ячейке вашего ноутбука. Они начинаются с символа восклицательного знака – !.

Далее !unzip распаковывает архив в папку с именем указанным после ключа -d (в нашем случае папка writers)

С помощью команды !ls мы выводим содержимое папки (`writers') на экран

Теперь все файлы для работы в вашем распоряжении!

In [ ]:
# Распаковка архива в папку 20writers
!unzip -qo 20writers.zip -d 20writers/

In [ ]:
# Просмотр содержимого папки
!ls 20writers

Беляев.txt    Гончаров.txt     Каверин.txt    Лесков.txt     Толстой.txt
Булгаков.txt  Горький.txt      Катаев.txt     Носов.txt      Тургенев.txt
Васильев.txt  Грибоедов.txt    Куприн.txt     Пастернак.txt  Чехов.txt
Гоголь.txt    Достоевский.txt  Лермонтов.txt  Пушкин.txt     Шолохов.txt


Важно! Если при распаковке файлов вы видите кракозябры вместо названий, необходимо указать кодировку:

!unzip -qo -O UTF-8 writers.zip -d writers/

Все три переменных ниже содержат названия строчного типа:

Первая – название папки с файлами текстов. Вторая – слово «обучающая» в названии файла (для поиска в дальнейшем). Третья – слово «тестовая» в названии файла (также для поиска).

In [ ]:
# Настройка констант для загрузки данных
FILE_DIR  = '20writers'                     # Папка с текстовыми файлами
SIG_TRAIN = 'обучающая'                   # Признак обучающей выборки в имени файла
SIG_TEST  = 'тестовая'                    # Признак тестовой выборки в имени файла



Первые три переменных содержат в себе по пустому списку, что вы видите по пустым квадратным скобкам – [ ].

● CLASS_LIST – в этот список вы поместите имя каждого класса и в итоге получите единый список с именами всех задействованных писателей;

● text_train – в этой переменной в результате будет список, содержащий одну единственную строку: непрерывный текст для обучения сети;

● text_test – аналогично text_train, но с назначением текста для тестирования точности сети.

Первый цикл for просто проходится по всем файлам с текстами в общей папке.

Команда re.match относится к разряду регулярных выражений, которые вы изучите отдельно, поскольку на первый взгляд они могут казаться весьма сложными.

Здесь re.match ищет совпадения в названиях файлов для деления по классам:

● class_name – это имя каждого писателя в названии файла.

● subset_name – наименование выборки (обучающая или тестовая).

re.match выполнит сортировку всех файлов. При этом данный метод ориентируется на строку '((.+)) (\S+)_' на первой позиции в своих параметрах. Это условно кодированные команды для работы с текстом и деления, которое как раз описано выше. Пока что примите этот параметр как данность, впоследствии вы сможете изучить его при желании.

Переменная cls сохраняет в себе список имен всех писателей в виде индексов, а первая и единственная в этом блоке команда print() информирует о распределении файлов по выборкам и создании списка с именами писателей.

Команда-менеджер with удобна для работы с потоком файлов, в вашем случае – с текстами писателей. Она гарантированно сократит количество строк для нужной операции до минимального, а также всегда поможет избежать возможных ошибок кода, которые могли бы случиться при использовании другого способа.

В блоке ниже with открывает каждый текстовый файл в папке и записывает весь текст в единую строку, сохраненную в переменной text. В переменной subset закреплена проверка принадлежности текста либо к обучающей, либо к проверочной выборке, и сразу после добавляет каждый текст к соответствующей выборке, разделив слова текста только пробелами.

len(CLASS_LIST) – сохраняет в переменной CLASS_COUNT общее количество писателей (их 20).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Загрузка датасета. Добавляются имена классов и соответствующие тексты.
# Все тексты преобразуются в строку и объединяются для каждого класса и выборки
CLASS_LIST = []
text_train = []
text_test = []

for file_name in os.listdir(FILE_DIR):
    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\) (\S+)_', file_name)
    # Если выделение получилось, то файл обрабатывается
    if m:
        class_name = m[1]
        subset_name = m[2].lower()
        # Проверка типа выборки в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая либо тестовая - файл обрабатывается
        if is_train or is_test:
            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)
                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:
                # Загрузка содержимого файла в строку
                text = f.read()
            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test
            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')

# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)

In [ ]:
# Прочитанные классы текстов
print(CLASS_LIST)

# Количество текстов в обучающей выборке
print(len(text_train))

# Количество символов в одном из текстов обучающей выборки
print(len(text_train[3]))

[]
0


IndexError: list index out of range

In [ ]:
# Проверим, были ли ошибки при загрузке текстов
for i, class_name in enumerate(CLASS_LIST):
    print(f"Количество текстов в обучающей выборке для класса '{class_name}': {len(text_train[i])}")

# Проверим ошибки при загрузке текстов
for i, class_name in enumerate(CLASS_LIST):
    print(f"Количество текстов в проверочной выборке для класса '{class_name}': {len(text_test[i])}")

# Проверим, были ли ошибки при загрузке текстов
for i, class_name in enumerate(CLASS_LIST):
    if len(text_train[i]) == 0:
        print(f"Ошибка при загрузке текстов для класса '{class_name}' в обучающей выборке")
    if len(text_test[i]) == 0:
        print(f"Ошибка при загрузке текстов для класса '{class_name}' в проверочной выборке")


In [ ]:
print("Количество классов:", len(CLASS_LIST))
print("Количество текстов в обучающей выборке:", len(text_train))
print("Количество текстов в проверочной выборке:", len(text_test))


Количество классов: 0
Количество текстов в обучающей выборке: 0
Количество текстов в проверочной выборке: 0


In [ ]:
# Загрузка датасета. Добавляются имена классов и соответствующие тексты.
CLASS_LIST = []
text_train = []
text_test = []

for file_name in os.listdir(FILE_DIR):
    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\)_(\S+)\.txt', file_name)
    # Если выделение получилось, то файл обрабатывается
    if m:
        class_name = m[1]
        subset_name = m[2].lower()
        # Проверка типа выборки в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая либо тестовая - файл обрабатывается
        if is_train or is_test:
            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)
                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:
                # Загрузка содержимого файла в строку
                text = f.read()
            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test
            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')

# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)


Давайте попробуем добавить больше отладочной информации, чтобы выявить, что именно происходит во время обработки файлов.

In [ ]:
# Загрузка датасета. Добавляются имена классов и соответствующие тексты.
CLASS_LIST = []
text_train = []
text_test = []

for file_name in os.listdir(FILE_DIR):
    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\)_(\S+)\.txt', file_name)
    # Вывод отладочной информации
    print(f"Имя файла: {file_name}, Результат выделения: {m}")
    # Если выделение получилось, то файл обрабатывается
    if m:
        class_name = m[1]
        subset_name = m[2].lower()
        # Вывод отладочной информации
        print(f"Имя класса: {class_name}, Тип выборки: {subset_name}")
        # Проверка типа выборки в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name
        # Вывод отладочной информации
        print(f"Обучающая выборка: {is_train}, Тестовая выборка: {is_test}")

        # Если тип выборки обучающая либо тестовая - файл обрабатывается
        if is_train or is_test:
            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)
                # Инициализация соответствующих классу строк текста
                text_train.append('')
                text_test.append('')

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:
                # Загрузка содержимого файла в строку
                text = f.read()
            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test
            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')

# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)


Имя файла: Тургенев.txt, Результат выделения: None
Имя файла: Носов.txt, Результат выделения: None
Имя файла: Куприн.txt, Результат выделения: None
Имя файла: Пастернак.txt, Результат выделения: None
Имя файла: Шолохов.txt, Результат выделения: None
Имя файла: Грибоедов.txt, Результат выделения: None
Имя файла: Чехов.txt, Результат выделения: None
Имя файла: Гончаров.txt, Результат выделения: None
Имя файла: Беляев.txt, Результат выделения: None
Имя файла: Катаев.txt, Результат выделения: None
Имя файла: Толстой.txt, Результат выделения: None
Имя файла: Горький.txt, Результат выделения: None
Имя файла: Достоевский.txt, Результат выделения: None
Имя файла: Каверин.txt, Результат выделения: None
Имя файла: Гоголь.txt, Результат выделения: None
Имя файла: Лесков.txt, Результат выделения: None
Имя файла: Булгаков.txt, Результат выделения: None
Имя файла: Пушкин.txt, Результат выделения: None
Имя файла: Васильев.txt, Результат выделения: None
Имя файла: Лермонтов.txt, Результат выделения: N

Давайте разделим каждый класс на обучающую и проверочную выборки. Мы можем использовать функцию train_test_split из библиотеки scikit-learn для этого. После разделения данных мы объединим их обратно в общие обучающие и проверочные выборки.

Давайте начнем с примера кода для разделения данных.

In [ ]:
from sklearn.model_selection import train_test_split

# Подготовим списки для обучающих и проверочных данных
X_train = []
X_test = []
y_train = []
y_test = []

# Процентное соотношение между обучающей и проверочной выборками
train_ratio = 0.8
test_ratio = 1 - train_ratio

for i, class_name in enumerate(CLASS_LIST):
    # Разделение данных для текущего класса на обучающую и проверочную выборки
    X_cls_train, X_cls_test = train_test_split(text_train[i], test_size=test_ratio, random_state=42)

    # Добавление разделенных данных в списки обучающих и проверочных данных
    X_train.extend(X_cls_train)
    X_test.extend(X_cls_test)
    y_train.extend([i] * len(X_cls_train))
    y_test.extend([i] * len(X_cls_test))

# Перемешивание данных
combined_train_data = list(zip(X_train, y_train))
combined_test_data = list(zip(X_test, y_test))
import random
random.shuffle(combined_train_data)
random.shuffle(combined_test_data)
X_train[:], y_train[:] = zip(*combined_train_data)
X_test[:], y_test[:] = zip(*combined_test_data)

# Проверка размеров обучающих и проверочных выборок
print("Размер обучающей выборки:", len(X_train))
print("Размер проверочной выборки:", len(X_test))


ValueError: not enough values to unpack (expected 2, got 0)

Похоже, что в процессе объединения текстов с классами что-то пошло не так. Давайте внимательно изучим код и исправим эту проблему.

Кажется, что списки text_train и text_test не содержат текстов для каждого класса. Давайте выведем их длины, чтобы убедиться, что данные загрузились правильно перед разделением на обучающую и проверочную выборки.

Давайте проверим длины списков text_train и text_test, чтобы убедиться, что данные были загружены корректно.

In [ ]:
# Проверка длины списков text_train и text_test
print("Длина списка text_train:", len(text_train))
print("Длина списка text_test:", len(text_test))


Длина списка text_train: 0
Длина списка text_test: 0


Понятно, это объясняет, почему произошла ошибка. Как мы видим, списки text_train и text_test оказались пустыми, что означает, что данные не были загружены правильно.

Давайте вернемся к коду, который загружает данные из файлов, и исправим проблему. Давайте убедимся, что регулярное выражение, которое мы использовали для извлечения имени класса и типа выборки из имен файлов, работает правильно. Для этого давайте снова выведем имена файлов и результаты выделения.

Давайте проверим регулярное выражение, которое мы использовали для извлечения имени класса и типа выборки из имен файлов. Для этого давайте выведем имена файлов и результаты выделения, чтобы убедиться, что регулярное выражение работает правильно.

In [ ]:
# Вывод имен файлов и результатов выделения
for file_name in os.listdir(FILE_DIR):
    # Выделение имени класса и типа выборки из имени файла
    m = re.match('\((.+)\)_(\S+)\.txt', file_name)
    print(f"Имя файла: {file_name}, Результат выделения: {m}")


Имя файла: Тургенев.txt, Результат выделения: None
Имя файла: Носов.txt, Результат выделения: None
Имя файла: Куприн.txt, Результат выделения: None
Имя файла: Пастернак.txt, Результат выделения: None
Имя файла: Шолохов.txt, Результат выделения: None
Имя файла: Грибоедов.txt, Результат выделения: None
Имя файла: Чехов.txt, Результат выделения: None
Имя файла: Гончаров.txt, Результат выделения: None
Имя файла: Беляев.txt, Результат выделения: None
Имя файла: Катаев.txt, Результат выделения: None
Имя файла: Толстой.txt, Результат выделения: None
Имя файла: Горький.txt, Результат выделения: None
Имя файла: Достоевский.txt, Результат выделения: None
Имя файла: Каверин.txt, Результат выделения: None
Имя файла: Гоголь.txt, Результат выделения: None
Имя файла: Лесков.txt, Результат выделения: None
Имя файла: Булгаков.txt, Результат выделения: None
Имя файла: Пушкин.txt, Результат выделения: None
Имя файла: Васильев.txt, Результат выделения: None
Имя файла: Лермонтов.txt, Результат выделения: N

Давайте попробуем использовать только часть до расширения .txt в имени файла как имя класса. Для этого мы можем разделить имя файла по точке и взять первую часть.

In [ ]:
# Вывод имен файлов и результатов выделения
for file_name in os.listdir(FILE_DIR):
    # Выделение имени класса из имени файла
    class_name = file_name.split('.')[0]
    print(f"Имя файла: {file_name}, Имя класса: {class_name}")


Имя файла: Тургенев.txt, Имя класса: Тургенев
Имя файла: Носов.txt, Имя класса: Носов
Имя файла: Куприн.txt, Имя класса: Куприн
Имя файла: Пастернак.txt, Имя класса: Пастернак
Имя файла: Шолохов.txt, Имя класса: Шолохов
Имя файла: Грибоедов.txt, Имя класса: Грибоедов
Имя файла: Чехов.txt, Имя класса: Чехов
Имя файла: Гончаров.txt, Имя класса: Гончаров
Имя файла: Беляев.txt, Имя класса: Беляев
Имя файла: Катаев.txt, Имя класса: Катаев
Имя файла: Толстой.txt, Имя класса: Толстой
Имя файла: Горький.txt, Имя класса: Горький
Имя файла: Достоевский.txt, Имя класса: Достоевский
Имя файла: Каверин.txt, Имя класса: Каверин
Имя файла: Гоголь.txt, Имя класса: Гоголь
Имя файла: Лесков.txt, Имя класса: Лесков
Имя файла: Булгаков.txt, Имя класса: Булгаков
Имя файла: Пушкин.txt, Имя класса: Пушкин
Имя файла: Васильев.txt, Имя класса: Васильев
Имя файла: Лермонтов.txt, Имя класса: Лермонтов


Прекрасно! Теперь, когда у нас есть правильные имена классов, давайте модифицируем наш код для загрузки данных и их разделения на обучающую и проверочную выборки. Мы также объединим их обратно в один набор данных для каждой выборки.

Посмотрите начало каждого текста из обучающей выборки:

In [ ]:
# Проверка загрузки: вывод начальных отрывков из каждого класса
for cls in range(CLASS_COUNT):
    print(f'Класс: {CLASS_LIST[cls]}')
    print(f'  train: {text_train[cls][:200]}')
    print(f'  test : {text_test[cls][:200]}')
    print()

NameError: name 'CLASS_COUNT' is not defined

Теперь вы узнаете, как превратить текст из последовательного набора слов в последовательный набор чисел.

Преобразование текстовых данных в числовые и векторные представления для обучения нейросети

понадобится более объемный словарь.

Это будет переменная VOCAB_SIZE, равная 20 000 индексам слов.

Так вы выбрали общий размер словаря, а посредством WIN_SIZE = 1000 вы задаете размер каждой подвыборки внутри общей базы. Так сколько же тогда вы получите подвыборок?

20 000 : 1000 = 20

20 примеров для обучения!

WIN_HOP = 100 – шаг или же попросту количество слов, на которое будет смещаться словарь от начала общего списка до самого конца, чтобы получить отличающиеся подвыборки.

In [ ]:
# Задание параметров преобразования
VOCAB_SIZE = 20000                        # Объем словаря для токенизатора
WIN_SIZE   = 1000                         # Длина отрезка текста (окна) в словах
WIN_HOP    = 100                          # Шаг окна разбиения текста на векторы

В блоке ниже создан класс timex, в основе которого лежит модуль для работы с реальным временем time.

timex используют для измерения времени выполнения дальнейших операций.

Важно: time.time() засекает текущее время, а {:.2f} в форматированной строке выводит время в виде десятичной дроби с двумя знаками (2f) после запятой.

In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

Далее выполняется та же операция по формированию словаря частотности, только его объем уже равен значению VOCAB_SIZE, а это 20 000.

Менеджер with поможет гладко обработать весь блок кода со словарем с помощью timex(). Так вы узнаете время обработки всего диапазона слов токенайзером. Учитывайте, что токенайзер еще не преобразует всю выборку сразу в одни лишь индексы (числовые значения), а формирует пары «слово: индекс».

In [ ]:
# Токенизация и построение частотного словаря по обучающим текстам
with timex():
    # Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                          lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)

    # Использованы параметры:
    # num_words   - объем словаря
    # filters     - убираемые из текста ненужные символы
    # lower       - приведение слов к нижнему регистру
    # split       - разделитель слов
    # char_level  - указание разделять по словам, а не по единичным символам
    # oov_token   - токен для слов, которые не вошли в словарь

    # Построение частотного словаря по обучающим текстам
    tokenizer.fit_on_texts(text_train)

    # Построение словаря в виде пар слово - индекс
    items = list(tokenizer.word_index.items())

Время обработки: 0.00 с


In [ ]:
# Вывод нескольких наиболее часто встречающихся слов
print(items[:120])

# Размер словаря может быть больше, чем num_words, но при преобразовании в последовательности
# и векторы bag of words будут учтены только первые num_words слов
print("Размер словаря", len(items))

[('неизвестное_слово', 1)]
Размер словаря 1


In [ ]:
# Проверка индекса слова в словаре
try:
    print('Интересующее слово имеет индекс:', tokenizer.word_index[input('Введите слово: ')])
except:
    print('Интересующего вас слова нет в словаре')

Введите слово: привет
Интересующего вас слова нет в словаре


А далее вы как раз преобразуете текст в чистую последовательность индексов по частоте повторений слов. Вы это рассматривали немного раньше в занятии.

In [ ]:
# Преобразование обучающих и проверочных текстов в последовательность индексов согласно частотному словарю
with timex():
    seq_train = tokenizer.texts_to_sequences(text_train)
    seq_test = tokenizer.texts_to_sequences(text_test)

    print("Фрагмент обучающего текста:")
    print("В виде оригинального текста:              ", text_train[1][:101])
    print("Он же в виде последовательности индексов: ", seq_train[1][:20])

Фрагмент обучающего текста:
В виде оригинального текста:               []
Он же в виде последовательности индексов:  []
Время обработки: 0.00 с


Рассмотрите функцию вывода статистики по всем загруженным текстам – print_text_stats(), которая:

Задает название выборки (параметр title – «обучающая» или «тестовая»). Отдельно проходит по каждому тексту (параметр texts). Проходит по сформированным последовательностям индексов слов (параметр sequences). Выводит количество символов каждого текста (счетчик переменной chars). Выводит количество слов каждого текста (счетчик переменной words). Суммирует все символы и слова в каждой выборке целиком (последняя строка в коде функции). Обратите внимание на вывод функции.

In [ ]:
# Функция вывода статистики по текстам
def print_text_stats(title, texts, sequences, class_labels=CLASS_LIST):
    # Суммарное количество символов и слов в тексте
    chars = 0
    words = 0

    print(f'Статистика по {title} текстам:')

    # Вывод итогов по всем классам данного набора текстов и их последовательностей индексов
    for cls in range(len(class_labels)):
        print('{:<15} {:9} символов,{:8} слов'.format(class_labels[cls],
                                                      len(texts[cls]),
                                                      len(sequences[cls])))
        chars += len(texts[cls])
        words += len(sequences[cls])

    print('----')
    print('{:<15} {:9} символов,{:8} слов\n'.format('В сумме', chars, words))

# Вывод итогов по текстам
print_text_stats('обучающим', text_train, seq_train)
print_text_stats('тестовым', text_test, seq_test)

Статистика по обучающим текстам:
Пушкин                  0 символов,       0 слов
Горький                 0 символов,       0 слов
Каверин                 0 символов,       0 слов
Катаев                  0 символов,       0 слов
Гоголь                  0 символов,       0 слов
Куприн                  0 символов,       0 слов
Гончаров                0 символов,       0 слов
Беляев                  0 символов,       0 слов
Носов                   0 символов,       0 слов
Чехов                   0 символов,       0 слов
Шолохов                 0 символов,       0 слов
Пастернак               0 символов,       0 слов
Лесков                  0 символов,       0 слов
Булгаков                0 символов,       0 слов
Тургенев                0 символов,       0 слов
Васильев                0 символов,       0 слов
Грибоедов               0 символов,       0 слов
Достоевский             0 символов,       0 слов
Лермонтов               0 символов,       0 слов
Толстой                 0 символов, 

Создание обучающей и проверочной выборок

Функция split_sequence поможет поделить одну общую последовательность (параметр sequence) из индексов слов на то количество примеров, которые вы захотите (параметр win_size) со сдвигом по всей выборке, равному размеру шага (параметр hop). Вы делали то же с помощью вложенного в список цикла for, только теперь он задан как результат работы функции. Кто знает, сколько раз вам придется к этому прибегать, поэтому лучше сделать функцию!

Есть и вторая функция – vectorize_sequence(), которая полезна двумя операциями:

деление общей последовательности на части с помощью функции split_sequence();

формирование бинарных (содержащих только 0 и 1) последовательностей верных ответов в соответствии с каждым примером для тренировочной и для тестовой выборок (метод to_categorical()).

Тогда в результате использования vectorize_sequence() вы всегда получите два готовых numpy-массива. Если подадите в функцию данные для обучения, то это будет обучающая выборка и выборка ответов для нее. То же будет и в случае с формированием массивов на основе тестовых данных.

In [ ]:
# Функция разбиения последовательности на отрезки скользящим окном
# На входе - последовательность индексов, размер окна, шаг окна
def split_sequence(sequence, win_size, hop):
    # Последовательность разбивается на части до последнего полного окна
    return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]


# Функция формирования выборок из последовательностей индексов
# формирует выборку отрезков и соответствующих им меток классов в виде one hot encoding
def vectorize_sequence(seq_list, win_size, hop):
    # В списке последовательности следуют в порядке их классов
    # Всего последовательностей в списке ровно столько, сколько классов
    class_count = len(seq_list)

    # Списки для исходных векторов и категориальных меток класса
    x, y = [], []

    # Для каждого класса:
    for cls in range(class_count):
        # Разбиение последовательности класса cls на отрезки
        vectors = split_sequence(seq_list[cls], win_size, hop)
        # Добавление отрезков в выборку
        x += vectors
        # Для всех отрезков класса cls добавление меток класса в виде OHE
        y += [utils.to_categorical(cls, class_count)] * len(vectors)

    # Возврат результатов как numpy-массивов
    return np.array(x), np.array(y)

Сформируйте выборки, посмотрите на их размер и заодно измерьте время выполнения операции. Последнее может отличаться в зависимости от среды выполнения и мощностей ПК:

In [ ]:
# Формирование обучающей и тестовой выборок
with timex():
    # Формирование обучающей выборки
    x_train, y_train = vectorize_sequence(seq_train, WIN_SIZE, WIN_HOP)
    # Формирование тестовой выборки
    x_test, y_test = vectorize_sequence(seq_test, WIN_SIZE, WIN_HOP)

    # Проверка формы сформированных данных
    print(x_train.shape, y_train.shape)
    print(x_test.shape, y_test.shape)

(0,) (0,)
(0,) (0,)
Время обработки: 0.00 с


Один из примеров последовательности индексов для обучения сети выглядит так:

In [ ]:
# Вывод отрезка индексов тренировочной выборки
print(x_train[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

Сформируйте отдельные обучающую и тестовую выборки в формате Bag Of Words, чтобы в дальнейшем использовать оба способа обработки текстов при обучении НС. И узнайте время выполнения этих двух операций:

In [ ]:
# Формирование выборок текстов в виде разреженных матриц (bag of words)
with timex():
    # На входе .sequences_to_matrix() ожидает список, .tolist() выполняет преобразование типа
    x_train_01 = tokenizer.sequences_to_matrix(x_train.tolist())
    x_test_01 = tokenizer.sequences_to_matrix(x_test.tolist())

    # Вывод формы обучающей выборки в виде разреженной матрицы Bag of Words
    print(x_train_01.shape)
    # Вывод фрагмента отрезка обучающего текста в виде Bag of Words
    print(x_train_01[0][0:100])

Различные архитектуры нейронных сетей для классификации текста Сервисные функции Пусть код ниже не вводит вас в ступор, там просто добавлена пара созданных функций для удобства компиляции и обучения модели. Именно это и будет делать функция под именем compile_train_model(). В ее параметры вы передадите все, что необходимо для компиляции и обучения:

● саму модель НС;

● выборки;

● функцию оптимизации;

● количество эпох обучения;

● размер шага по выборке;

● размер полотна графика.

Да, функция даже нарисует для вас графики точности и ошибки сети! Не зря вы познакомились с библиотекой matplotlib (блок кода по отрисовке графиков последний в данной функции).

Также в ячейке ниже написана функция eval_model(). Она оценивает точность работы сети при распознавании текстов, дает статистику по верно и неверно распознанным классам, и рисует график-матрицу ошибки предсказания сети! Это очень красивый график, похожий на яркую шахматную доску. Как его правильно читать, вы узнаете чуть позже.

Последняя функция compile_train_eval_model() объединяет в себе функционал двух предыдущих. Ей вы и будете пользоваться, чтобы в будущем строить разные архитектуры, при этом в одну строку задавая компиляцию модели, обучение и оценку точности!

In [ ]:
# Функция компиляции и обучения модели нейронной сети
def compile_train_model(model,
                        x_train,
                        y_train,
                        x_val,
                        y_val,
                        optimizer='adam',
                        epochs=50,
                        batch_size=128,
                        figsize=(20, 5)):

    # Компиляция модели
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Вывод сводки
    model.summary()

    # Обучение модели с заданными параметрами
    history = model.fit(x_train,
                        y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_val, y_val))

    # Вывод графиков точности и ошибки
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('График процесса обучения модели')
    ax1.plot(history.history['accuracy'],
               label='Доля верных ответов на обучающем наборе')
    ax1.plot(history.history['val_accuracy'],
               label='Доля верных ответов на проверочном наборе')
    ax1.xaxis.get_major_locator().set_params(integer=True)
    ax1.set_xlabel('Эпоха обучения')
    ax1.set_ylabel('Доля верных ответов')
    ax1.legend()

    ax2.plot(history.history['loss'],
               label='Ошибка на обучающем наборе')
    ax2.plot(history.history['val_loss'],
               label='Ошибка на проверочном наборе')
    ax2.xaxis.get_major_locator().set_params(integer=True)
    ax2.set_xlabel('Эпоха обучения')
    ax2.set_ylabel('Ошибка')
    ax2.legend()
    plt.show()


# Функция вывода результатов оценки модели на заданных данных
def eval_model(model, x, y_true,
               class_labels=[],
               cm_round=3,
               title='',
               figsize=(10, 10)):
    # Вычисление предсказания сети
    y_pred = model.predict(x)
    # Построение матрицы ошибок
    cm = confusion_matrix(np.argmax(y_true, axis=1),
                          np.argmax(y_pred, axis=1),
                          normalize='true')
    # Округление значений матрицы ошибок
    cm = np.around(cm, cm_round)

    # Отрисовка матрицы ошибок
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title(f'Нейросеть {title}: матрица ошибок нормализованная', fontsize=18)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
    disp.plot(ax=ax)
    plt.gca().images[-1].colorbar.remove()  # Стирание ненужной цветовой шкалы
    plt.xlabel('Предсказанные классы', fontsize=16)
    plt.ylabel('Верные классы', fontsize=16)
    fig.autofmt_xdate(rotation=45)          # Наклон меток горизонтальной оси при необходимости
    plt.show()

    print('-'*100)
    print(f'Нейросеть: {title}')

    # Для каждого класса:
    for cls in range(len(class_labels)):
        # Определяется индекс класса с максимальным значением предсказания (уверенности)
        cls_pred = np.argmax(cm[cls])
        # Формируется сообщение о верности или неверности предсказания
        msg = 'ВЕРНО :-)' if cls_pred == cls else 'НЕВЕРНО :-('
        # Выводится текстовая информация о предсказанном классе и значении уверенности
        print('Класс: {:<20} {:3.0f}% сеть отнесла к классу {:<20} - {}'.format(class_labels[cls],
                                                                               100. * cm[cls, cls_pred],
                                                                               class_labels[cls_pred],
                                                                               msg))

    # Средняя точность распознавания определяется как среднее диагональных элементов матрицы ошибок
    print('\nСредняя точность распознавания: {:3.0f}%'.format(100. * cm.diagonal().mean()))


# Совместная функция обучения и оценки модели нейронной сети
def compile_train_eval_model(model,
                             x_train,
                             y_train,
                             x_test,
                             y_test,
                             class_labels=CLASS_LIST,
                             title='',
                             optimizer='adam',
                             epochs=50,
                             batch_size=128,
                             graph_size=(20, 5),
                             cm_size=(10, 10)):
    # Компиляция и обучение модели на заданных параметрах
    # В качестве проверочных используются тестовые данные
    compile_train_model(model,
                        x_train, y_train,
                        x_test, y_test,
                        optimizer=optimizer,
                        epochs=epochs,
                        batch_size=batch_size,
                        figsize=graph_size)

    # Вывод результатов оценки работы модели на тестовых данных
    eval_model(model, x_test, y_test,
               class_labels=class_labels,
               title=title,
               figsize=cm_size)

Попробуйте способ обучения сети на Bag Of Words. Сеть будет угадывать одного из шести авторов для каждого примера текста. На выходном слое нужно указать функцию активации softmax.

Соберите полносвязную модель, подобную той, что вы уже рассматривали на первом занятии по НС. А на обучение и тест сети подайте выборки формата Bag Of Words (x_train_01 и x_test_01), которые вы подготовили ранее.

Функция compile_train_eval_model() также выведет статистику по сформированной модели до начала обучения. Смотрите:

In [ ]:
# Создание последовательной модели нейросети
model_text_bow_softmax = Sequential()
# Первый полносвязный слой
model_text_bow_softmax.add(Dense(200, input_dim=VOCAB_SIZE, activation="relu"))
# Слой регуляризации Dropout
model_text_bow_softmax.add(Dropout(0.25))
# Слой пакетной нормализации
model_text_bow_softmax.add(BatchNormalization())
#Добавим второй полносвязный слой
model_text_bow_softmax.add(Dense(100, input_dim=VOCAB_SIZE, activation="relu"))
# Еще один слой регуляризации Dropout
model_text_bow_softmax.add(Dropout(0.15))
#Добавляем еще один денс-слой
model_text_bow_softmax.add(Dense(50, input_dim=VOCAB_SIZE, activation="relu"))

# Выходной полносвязный слой
model_text_bow_softmax.add(Dense(CLASS_COUNT, activation='softmax'))

# Входные данные подаются в виде векторов bag of words
compile_train_eval_model(model_text_bow_softmax,
                         x_train_01, y_train,
                         x_test_01, y_test,
                         class_labels=CLASS_LIST,
                         title='BoW')

In [ ]:
# Сохранение полной модели (архитектуры слоев и весов каждого слоя)
# Для чтения модели используется метод keras.models.load_model()
model_text_bow_softmax.save('model_text_bow_softmax.h5')

Embedding. Как вы помните, этот слой создает из каждого индекса слова отдельный вектор числовых значений, размер которого равен параметру, указываемому при создании слоя в архитектуре.

Первая модель Embedding создаст для каждого слова в выборке вектор из 20 значений. Размер вектора, который создает слой, указывается в параметре output_dim. Это довольно малый размер, сеть обучится быстро, но и точность может оказаться невысокой. Параметр input_dim указывает количество уникальных значений (токенов), который данных слой будет способен обработать (если мы установим значение input_dim = N и подадим токен больше чем N, этот слой выдаст ошибку, по скольку у него не будут нужные веса для обработки этого токена).

Для работы с Embedding вы подадите выборки, созданные до преобразования в Bag Of Words – просто x_train и x_test:

Embedding + Dense: размерность эмбеддингов 20

In [ ]:
# Архитектура со слоем Embedding и регуляризацией
model_text_emb_20 = Sequential()
model_text_emb_20.add(Embedding(input_dim = VOCAB_SIZE, output_dim = 20, input_length=WIN_SIZE))
model_text_emb_20.add(SpatialDropout1D(0.2))
model_text_emb_20.add(Flatten())
model_text_emb_20.add(BatchNormalization())
model_text_emb_20.add(Dense(200, activation="relu"))
model_text_emb_20.add(Dropout(0.2))
model_text_emb_20.add(BatchNormalization())
model_text_emb_20.add(Dense(CLASS_COUNT, activation='softmax'))

# Входные данные подаются в виде последовательностей индексов,
# а не векторов bag of words
compile_train_eval_model(model_text_emb_20,
                         x_train, y_train,
                         x_test, y_test,
                         class_labels=CLASS_LIST,
                         title='Embedding/20')

Выводы по итогам обучения модели Embedding(20) + Dense

Посмотрите на графики и данные по эпохам. Точность и ошибка на проверочном наборе нарисовали пилу, хотя на обучающем по-прежнему полный штиль. Однако график матрицы ошибок не даст вам себя обмануть: вы видите, как изменилась тональность на диагонали и как упали значения точности. И правда, на последней эпохе сеть дала точность распознавания 70%. Результат модели Embedding пока что сильно уступает Bag Of Words.

Посмотрите, как изменится точность, если увеличить размер Embedding-пространства до 200, а также добавить новый слой одномерной регуляризации SpatialDropout1D.

Embedding + Dense: размерность эмбеддингов 200

In [ ]:
#Создаём сеть с Embedding слоем
model_text_emb_200 = Sequential()
model_text_emb_200.add(Embedding(VOCAB_SIZE, 200, input_length=WIN_SIZE))
model_text_emb_200.add(SpatialDropout1D(0.2))
model_text_emb_200.add(Flatten())
model_text_emb_200.add(BatchNormalization())
model_text_emb_200.add(Dense(200, activation="relu"))
model_text_emb_200.add(Dropout(0.2))
model_text_emb_200.add(BatchNormalization())
model_text_emb_200.add(Dense(CLASS_COUNT, activation='softmax'))

compile_train_eval_model(model_text_emb_200,
                         x_train, y_train,
                         x_test, y_test,
                         class_labels=CLASS_LIST,
                         title='Embedding/200')